In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For animation
from matplotlib.animation import FuncAnimation
from celluloid import Camera

In [ ]:
df = pd.read_csv("dataset_spiketimes_model1d.dat", sep = '\t', header = None)


data_start = 10.4 # in s
data_end = 13.4 # in s

df = df.drop(df[df[0] < data_start].index)
df = df.drop(df[df[0] > data_end].index)

df.index = range(len(df[0])) # recreate indexes from 0 to ...

df[0] = df[0].round(decimals = 4)

In [ ]:
%matplotlib notebook

In [ ]:
plt.figure()
plt.plot(df[0],df[1],'.')
# plt.xlim([10,20])
plt.show()

In [ ]:
# Animation

movie_duration = 2
fps = 1

len_data = len(df[0])


fig = plt.figure()

camera = Camera(fig)
frame_density = (len_data / (movie_duration * fps))

weights_colormap = plt.get_cmap("Blues")


density = 100

for i in range(0,len_data-(density),density):
    
    plt.plot(df[0][0:(i+density)], df[1][0:(i+density)], '.', color = weights_colormap(30))
    plt.plot(df[0][i:(i+density)], df[1][i:(i+density)], '.', color = weights_colormap(200))

    camera.snap()

In [ ]:
animation = camera.animate()

In [ ]:
# animation.save('test1D.mp4', dpi = 300, fps = fps)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

In [ ]:
# Arnau Manasanch (github: @arnaumanasanch) contribution: 

%matplotlib notebook
import ipywidgets as widgets
import asyncio
import matplotlib.pyplot as plt
steps = 200
tail = 3
step_times = np.linspace(df[0].values[0], df[0].values[-1], steps)
dt = np.mean(np.diff(step_times))
fig, ax = plt.subplots(figsize=(10, 6))
points = ax.scatter([],[],c='tab:blue', marker='.')
ax.set_xlim([df[0].values[0], df[0].values[-1]])
ax.set_ylim([0, 1024])
for t in step_times:
    # get neurons within the tail time
    df_tail = df.loc[(df[0]>t-tail) & (df[0]<=t), [0, 1]]
    neurons = np.zeros(len(df_tail))
    for n in range(len(df_tail)):
        neurons[n] = t - df_tail[0].iloc[n]
    if np.sum(neurons) == 0:
        neurons += 1
    else:
        neurons = (neurons - np.min(neurons)) / (np.max(neurons) - np.min(neurons))
    neurons = 1-neurons
    neurons[neurons<0.2] = 0.2
    points.set_offsets(np.array([df_tail[0].values, df_tail[1].values]).T)
    points.set_alpha(neurons)
#     ax.set_title(t)
    fig.canvas.draw()
    plt.pause(0.05)